In [25]:
__all__ = [
    "IBConnector",
    "IBCallbacks",
    "IBRequests",
    "LoggingConfig",
    "PACKAGE_NAME",
    "VERSION",
]

In [26]:
type(__all__)

list

In [33]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi import wrapper
from ibapi.common import *  # Includes many shared datatypes
from ibapi.ticktype import *  # For tick data
from ibapi.contract import Contract
from ibapi.contract import ContractDetails

In [13]:
import logging
import json

In [12]:
import datetime
import pytz
import pandas as pd
import numpy as np

In [21]:
ls

historical_analysis.ipynb  main.ipynb
ibapi_demo_test.ipynb      realtime_analysis.ipynb
log_file.log               trading_strategy.ipynb


In [34]:
# Custom JSON formatter for logs
class JsonFormatter(logging.Formatter):
    def format(self, record):
        # Initialize log record with timestamp and level first
        log_record = {
            "timestamp": self.formatTime(record, self.datefmt),
            "level": record.levelname
        }

        # If the message is a dictionary, merge it into the log record
        if isinstance(record.msg, dict):
            log_record.update(record.msg)
        else:
            # Fallback for non-dictionary messages
            log_record["message"] = record.getMessage()

        # Return the JSON-formatted string with indentation
        return json.dumps(log_record, indent=2)

# Configure the logger
# log_file_path = '/Volumes/LaCie_d2_Professional_Media/ibapi_callbacks/callbacks.log'
log_file_path = 'log_file.log' #local

file_handler = logging.FileHandler(log_file_path)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(JsonFormatter())

logger = logging.getLogger("JsonLogger")
logger.setLevel(logging.INFO)

# Ensure no duplicate handlers
if not logger.hasHandlers():
    logger.addHandler(file_handler)

# Example usage for your application logs
trade_app_logger = logging.getLogger('TradeApp')

# Ensure TradeApp logger uses the same handlers
trade_app_logger.setLevel(logging.INFO)
trade_app_logger.addHandler(file_handler)

# Example log
trade_app_logger.info("IBAPI log initialized.")

### Create callback functions here to handle responses from API

In [35]:
class TradeApp(EWrapper, EClient):
    def __init__(self, json_indent=2):
        EWrapper.__init__(self)
        EClient.__init__(self, self)
        self.json_indent = json_indent  # Control JSON indentation
        self.reqId_to_symbol = {}  # Map reqId to symbols for easier tracking

    
    def accountSummary(self, reqId: int, account: str, tag: str, value: str, currency: str):
        print(f"Account Summary Update - Request ID: {reqId}, Account: {account}, Tag: {tag}, Value: {value}, Currency: {currency}")

    def accountSummaryEnd(self, reqId: int):
        print(f"Account Summary End - Request ID: {reqId}")

    def contractDetails(self, reqId: int, contractDetails: ContractDetails):
        print(f"Contract Details for Request ID {reqId}:")
        print(f"Symbol: {contractDetails.contract.symbol}")
        print(f"Exchange: {contractDetails.contract.exchange}")
        print(f"Currency: {contractDetails.contract.currency}")
        print(f"Last Trade Date: {contractDetails.contract.lastTradeDateOrContractMonth}")
        print(f"Multiplier: {contractDetails.contract.multiplier}")
        print(f"Trading Hours: {contractDetails.tradingHours}")
        print(f"Liquid Hours: {contractDetails.liquidHours}")
        print(f"Price Magnifier: {contractDetails.priceMagnifier}")

    def contractDetailsEnd(self, reqId: int):
        """Called when all contract details are received."""
        print(f"Contract Details End - Request ID: {reqId}")        
    
    def log_and_print(self, log_msg):
        """Helper method to log and optionally print JSON-formatted messages."""
        logger.info(log_msg)
        if PrintInLine:
            print(json.dumps(log_msg, indent=self.json_indent))

    def tickPrice(self, reqId: int, tickType: int, price: float, attrib):
        """Capture tick price, including margin-related fields."""
        if tickType == 87:  # Field ID for margin-related tick data
            log_msg = {
                "type": "MarginData",
                "details": {
                    "symbol": self.reqId_to_symbol.get(reqId, "Unknown"),
                    "margin": price,
                    "tickType": tickType
                }
            }
            self.log_and_print(log_msg)

    def tickSize(self, reqId: int, tickType: int, size: int):
        """Capture tick size if needed."""
        log_msg = {
            "type": "TickSize",
            "details": {
                "reqId": reqId,
                "tickType": tickType,
                "size": size
            }
        }
        self.log_and_print(log_msg)

    def tickSnapshotEnd(self, reqId: int):
        """Triggered when snapshot data is complete."""
        log_msg = {
            "type": "TickSnapshotEnd",
            "details": {"reqId": reqId}
        }
        self.log_and_print(log_msg)

### Initiate thread for jupyter kernel communication to API

In [41]:
import time
import threading
from ibapi.client import EClient
from ibapi.wrapper import EWrapper


class TradeApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)

    def error(self, reqId, errorCode, errorString):
        print("Error:", reqId, errorCode, errorString)

    def nextValidId(self, orderId):
        super().nextValidId(orderId)
        print("Connected. Your next valid order ID is:", orderId)
        self.nextValidOrderId = orderId


def run_loop(app):
    app.run()


app = TradeApp()
app.connect("127.0.0.1", 7497, clientId=2)

api_thread = threading.Thread(target=run_loop, args=(app,), daemon=True)
api_thread.start()

# Give IBAPI a moment to connect before proceeding
time.sleep(2)

print("Is connected?", app.isConnected())


Connected. Your next valid order ID is: 1
Error: -1 2104 Market data farm connection is OK:usfarm.nj
Error: -1 2104 Market data farm connection is OK:usfuture
Error: -1 2104 Market data farm connection is OK:cashfarm
Error: -1 2104 Market data farm connection is OK:usfarm
Error: -1 2106 HMDS data farm connection is OK:ushmds
Error: -1 2158 Sec-def data farm connection is OK:secdefil
Is connected? True


In [42]:
import threading #Jupyter Notebook Execution: If app.run() blocks the Jupyter kernel, you can use threading

app = TradeApp()
def run_loop():
    app.run()
app.connect("127.0.0.1", 7497, clientId=2)

# Start the API in a separate thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

if PrintInLine:
    print("IBAPI log initialized to this Jupyter thread.")

NameError: name 'PrintInLine' is not defined

# IBAPI Account Request Tags
*Requests use the built-in API calls. Be sure to have response handling by defining callback functions in class `TradeApp`*

These tags provide essential account and margin details critical for day trading and algo trading in **equities** and **futures**:

## Core Account Metrics
- **`accountType`**: Type of account (e.g., Cash, Margin).
- **`NetLiquidation`**: Total account value (cash + positions).
- **`TotalCashValue`**: Cash balance, including unrealized futures PnL.
- **`SettledCash`**: Settled cash available (same as `TotalCashValue` for cash accounts).
- **`AccruedCash`**: Net accrued interest.

## Buying Power & Margin
- **`BuyingPower`**: Maximum marginable amount for US stocks.
- **`EquityWithLoanValue`**: Account equity inclusive of loan/margin.
- **`RegTEquity`**: Regulation T equity requirement.
- **`RegTMargin`**: Regulation T margin requirement.
- **`SMA`**: Special Memorandum Account (used for calculating buying power).
- **`InitMarginReq`**: Initial margin requirement for current positions.
- **`MaintMarginReq`**: Maintenance margin requirement.

## Risk Management
- **`AvailableFunds`**: Funds available for trading after margin.
- **`ExcessLiquidity`**: Cushion amount before liquidation risks.
- **`Cushion`**: Percentage of excess liquidity to net liquidation value.
- **`HighestSeverity`**: Severity level of nearing liquidation.

## Look-Ahead Metrics
- **`LookAheadInitMarginReq`**, **`LookAheadMaintMarginReq`**: Predicted margin requirements.
- **`LookAheadAvailableFunds`**, **`LookAheadExcessLiquidity`**: Predicted available funds and liquidity.
- **`LookAheadNextChange`**: Time of next look-ahead margin update.

## Trading-Specific
- **`GrossPositionValue`**: Total absolute value of all positions.
- **`DayTradesRemaining`**: Number of allowed day trades before Pattern Day Trader (PDT) detection (`-1` means unlimited).
- **`Leverage`**: Ratio of `GrossPositionValue` to `NetLiquidation`.

## Ledger (Cash Balances)
- **`$LEDGER`**: All cash balances in base currency.
- **`$LEDGER:CURRENCY`**: Cash balances in a specific currency.
- **`$LEDGER:ALL`**: Cash balances across all currencies.

---

## Why These Matter
- **Risk Management**: Use `ExcessLiquidity`, `Cushion`, and `HighestSeverity` to monitor liquidation risk.
- **Margin Awareness**: Track `BuyingPower`, `InitMarginReq`, and `MaintMarginReq` for margin-sensitive strategies.
- **Day Trading Rules**: Monitor `DayTradesRemaining` to stay compliant with PDT rules.
- **Capital Allocation**: Utilize `TotalCashValue` and `NetLiquidation` for proper strategy allocation.
- **Position Scaling**: Use `Leverage` and `GrossPositionValue` for controlling portfolio size.

---

This information helps ensure you maintain proper account risk management and optimize your trading strategies programmatically.

In [33]:
app.reqAccountSummary(9001, "All", 'TotalCashValue,DayTradesRemaining,BuyingPower,Cushion') # creates a subscription your account data, refreshing periodically and updating in real-time while the subscription is active.

# app.cancelAccountSummary(9001) #closes out the subscription

In [17]:
app.reqAccountUpdates(True, "All")

In [13]:
app.disconnect()

In [44]:
def create_futures_contract(symbol, expiry, exchange="GLOBEX", currency="USD"):
    contract = Contract()
    contract.symbol = symbol        # e.g., "MES"
    contract.secType = "FUT"        # Futures
    contract.exchange = exchange    # "GLOBEX"
    contract.currency = currency    # "USD"
    contract.lastTradeDateOrContractMonth = expiry  # e.g., "202503" for March 2025
    return contract

# Example usage
futures_contract = create_futures_contract("CL", "202503", "NYMEX")


# Request contract details (including margins)
app.reqContractDetails(9002, futures_contract)

In [ ]:
app.reqContractDetails(9003,)

In [40]:
futures_contract

4465945536: 0,MES,FUT,202503,,0,,,GLOBEX,,USD,,,False,,,,combo:

In [60]:
app.reqPositions()

### disconnect from server here

In [18]:
app.disconnect()

In [43]:
import time
import threading
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract


class MyWrapper(EWrapper):
    def __init__(self):
        super().__init__()
        self.historical_data = []

    def historicalData(self, reqId, bar):
        """
        Called by the API for each bar of historical data.
        """
        print(f"{reqId} - Date: {bar.date}, Open: {bar.open}, High: {bar.high}, "
              f"Low: {bar.low}, Close: {bar.close}, Volume: {bar.volume}")
        self.historical_data.append(bar)

    def historicalDataEnd(self, reqId, start, end):
        """
        Called when historical data download is complete.
        """
        print(f"Finished downloading historical data for request {reqId}. "
              f"Start: {start}, End: {end}")


class MyClient(EClient):
    def __init__(self, wrapper):
        EClient.__init__(self, wrapper)


class MyApp(MyWrapper, MyClient):
    def __init__(self):
        MyWrapper.__init__(self)
        MyClient.__init__(self, wrapper=self)
        self.req_id = 1  # You can change or increment this as needed

    def error(self, reqId, errorCode, errorString):
        """
        Called to notify of an error.
        """
        super().error(reqId, errorCode, errorString)
        print(f"Error. ID: {reqId}, Code: {errorCode}, Msg: {errorString}")


def run_loop(app):
    """
    The worker thread that continuously processes incoming messages.
    """
    app.run()


def main():
    app = MyApp()
    app.connect("127.0.0.1", 7497, clientId=123)  # Replace with the correct port if needed

    # Start the socket in a new thread
    api_thread = threading.Thread(target=run_loop, args=(app,), daemon=True)
    api_thread.start()

    # Give the connection a few seconds to establish
    time.sleep(2)

    # Create a contract object for Apple
    apple_contract = Contract()
    apple_contract.symbol = "AAPL"
    apple_contract.secType = "STK"
    apple_contract.exchange = "SMART"
    apple_contract.currency = "USD"

    # Request historical data
    """
    - endDateTime: '' (empty) means "now"
    - durationStr: '1 Y' means 1 year. Examples: '1 D', '1 W', '1 M', '1 Y', etc.
    - barSizeSetting: '1 day' means daily bars. Options include '1 min', '5 mins', '1 hour', etc.
    - whatToShow: 'TRADES' or 'MIDPOINT' or 'BID'/'ASK'
    - useRTH: 1 means regular trading hours only; 0 means all data including pre/post-market
    - formatDate: 1 returns date in 'YYYYMMDD' format
    """
    app.reqHistoricalData(
        reqId=app.req_id,
        contract=apple_contract,
        endDateTime='',
        durationStr='1 Y',
        barSizeSetting='1 day',
        whatToShow='TRADES',
        useRTH=1,
        formatDate=1,
        keepUpToDate=False,
        chartOptions=[]
    )

    # Let the program run long enough to receive all data
    time.sleep(10)

    # Disconnect
    app.disconnect()


if __name__ == "__main__":
    main()


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


Error. ID: -1, Code: 2104, Msg: Market data farm connection is OK:usfarm.nj
Error. ID: -1, Code: 2104, Msg: Market data farm connection is OK:usfuture
Error. ID: -1, Code: 2104, Msg: Market data farm connection is OK:cashfarm
Error. ID: -1, Code: 2104, Msg: Market data farm connection is OK:usfarm
Error. ID: -1, Code: 2106, Msg: HMDS data farm connection is OK:ushmds
Error. ID: -1, Code: 2158, Msg: Sec-def data farm connection is OK:secdefil


ERROR 1 2176 Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 368883.69 to 368883


Error. ID: 1, Code: 2176, Msg: Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 368883.69 to 368883
1 - Date: 20240125, Open: 195.22, High: 196.27, Low: 193.11, Close: 194.17, Volume: 368883
1 - Date: 20240126, Open: 194.26, High: 194.76, Low: 191.94, Close: 192.42, Volume: 287589
1 - Date: 20240129, Open: 192.01, High: 192.2, Low: 189.58, Close: 191.73, Volume: 282914
1 - Date: 20240130, Open: 190.95, High: 191.8, Low: 187.47, Close: 188.04, Volume: 384851
1 - Date: 20240131, Open: 187.13, High: 187.15, Low: 184.4, Close: 184.4, Volume: 324014
1 - Date: 20240201, Open: 184.01, High: 186.95, Low: 183.81, Close: 186.86, Volume: 346937
1 - Date: 20240202, Open: 179.87, High: 187.33, Low: 179.25, Close: 185.85, Volume: 684671
1 - Date: 20240205, Open: 188.13, High: 189.25, Low: 185.84, Close: 187.68, Volume: 450294
1 - Date: 20240206, Open: 186.9, High: 189.31, Low: 186.76, Close: 189.3, Volume: 289554
1 - Date:

In [45]:
import time
import threading
import pandas as pd
import json

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract


class MyWrapper(EWrapper):
    def __init__(self):
        super().__init__()
        self.historical_bars = []
        self.data_downloaded = False  # Flag to signal when download is complete

    def historicalData(self, reqId, bar):
        """
        Called for each bar of historical data.
        """
        self.historical_bars.append({
            'date': bar.date,
            'open': bar.open,
            'high': bar.high,
            'low': bar.low,
            'close': bar.close,
            'volume': bar.volume
        })

    def historicalDataEnd(self, reqId, start, end):
        """
        Called when historical data download is finished.
        """
        print(f"Historical data download completed. Start: {start}, End: {end}")
        self.data_downloaded = True

    def error(self, reqId, errorCode, errorString):
        """
        Capture errors thrown by IB API.
        """
        print(f"Error. ReqId={reqId}, Code={errorCode}, Msg={errorString}")


class MyClient(EClient):
    def __init__(self, wrapper):
        super().__init__(wrapper)


class MyApp(MyWrapper, MyClient):
    def __init__(self):
        MyWrapper.__init__(self)
        MyClient.__init__(self, wrapper=self)


def run_loop(app):
    """
    The IB API listener thread that processes incoming messages.
    """
    app.run()


def get_apple_historical_data(
    output_format="df",
    host="127.0.0.1",
    port=7497,
    client_id=0,
    duration="1 Y",
    bar_size="1 day"
):
    """
    Connect to Interactive Brokers (TWS or IB Gateway) and request Apple
    historical data. Return the data as either a Pandas DataFrame or JSON.

    Args:
        output_format (str): 'df' returns a Pandas DataFrame, 'json' returns JSON.
        host (str): TWS/IB Gateway host (default '127.0.0.1').
        port (int): TWS/IB Gateway port (default 7497 for paper trading).
        client_id (int): Unique client ID. Must differ from other connected apps.
        duration (str): Historical data duration (e.g. '1 D', '1 W', '1 M', '1 Y').
        bar_size (str): Bar size (e.g. '1 min', '5 mins', '1 hour', '1 day').

    Returns:
        Pandas DataFrame if output_format == 'df',
        or JSON string if output_format == 'json'.
    """

    # 1) Instantiate and connect the app
    app = MyApp()
    app.connect(host, port, client_id)

    # 2) Start the IB API network loop in a separate daemon thread
    api_thread = threading.Thread(target=run_loop, args=(app,), daemon=True)
    api_thread.start()

    # Give the connection a moment to initialize
    time.sleep(2)

    # 3) Create Apple Contract
    contract = Contract()
    contract.symbol = "AAPL"
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"

    # 4) Request historical data
    #    - endDateTime='' means "current time"
    #    - durationStr='1 Y' = 1 year
    #    - barSizeSetting='1 day' = daily bars
    #    - whatToShow='TRADES' for trade prices
    #    - useRTH=1 means only regular trading hours
    #    - keepUpToDate=False means snapshot, not streaming data
    app.reqHistoricalData(
        reqId=1,
        contract=contract,
        endDateTime='',
        durationStr=duration,
        barSizeSetting=bar_size,
        whatToShow='TRADES',
        useRTH=1,
        formatDate=1,
        keepUpToDate=False,
        chartOptions=[]
    )

    # 5) Wait (up to 30 seconds) for data to complete downloading
    start_time = time.time()
    while not app.data_downloaded and (time.time() - start_time) < 30:
        time.sleep(1)

    # 6) Disconnect
    app.disconnect()

    # 7) Return the data in requested format
    if output_format.lower() == "df":
        return pd.DataFrame(app.historical_bars)
    elif output_format.lower() == "json":
        # Dump as JSON (list of records)
        return json.dumps(app.historical_bars, indent=4)
    else:
        raise ValueError("Invalid output_format. Use 'df' or 'json'.")


# ---------------
# Example usage:
# ---------------
if __name__ == "__main__":
    # Return data as a Pandas DataFrame
    aapl_df = get_apple_historical_data(output_format="df")
    print("Returned DataFrame:")
    print(aapl_df.head())

    # Return data as JSON
    aapl_json = get_apple_historical_data(output_format="json")
    print("\nReturned JSON:")
    print(aapl_json)


Error. ReqId=-1, Code=2104, Msg=Market data farm connection is OK:usfarm.nj
Error. ReqId=-1, Code=2104, Msg=Market data farm connection is OK:usfuture
Error. ReqId=-1, Code=2104, Msg=Market data farm connection is OK:cashfarm
Error. ReqId=-1, Code=2104, Msg=Market data farm connection is OK:usfarm
Error. ReqId=-1, Code=2106, Msg=HMDS data farm connection is OK:ushmds
Error. ReqId=-1, Code=2158, Msg=Sec-def data farm connection is OK:secdefil
Error. ReqId=1, Code=2176, Msg=Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 368883.69 to 368883
Historical data download completed. Start: 20240123  21:32:12, End: 20250123  21:32:12
Returned DataFrame:
       date    open    high     low   close  volume
0  20240125  195.22  196.27  193.11  194.17  368883
1  20240126  194.26  194.76  191.94  192.42  287589
2  20240129  192.01  192.20  189.58  191.73  282914
3  20240130  190.95  191.80  187.47  188.04  384851
4  202401